# Debug & Fix Qwen3-Reranker-8B

Re-rankers SHOULD outperform embeddings for:
- Ambiguous queries
- Context-dependent questions
- Precise keyword matching

Let's find and fix why yours isn't working correctly.

In [ ]:
import yaml
import json
import boto3
import numpy as np

with open('config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("✓ Config loaded")

In [ ]:
# =============================================================================
# RERANKER ENDPOINT CONFIG
# =============================================================================
RERANKER_ENDPOINT = 'your-qwen3-reranker-endpoint'  # <-- UPDATE THIS

reranker_client = boto3.client(
    'sagemaker-runtime',
    region_name=config['models']['embedding']['credentials']['region'],
    aws_access_key_id=config['models']['embedding']['credentials']['accessKeyId'],
    aws_secret_access_key=config['models']['embedding']['credentials']['secretAccessKey'],
    aws_session_token=config['models']['embedding']['credentials']['sessionToken']
)

print(f"✓ Client ready for endpoint: {RERANKER_ENDPOINT}")

## Step 1: Test Different Input Formats

Different inference engines expect different formats. Let's find what works.

In [ ]:
# Test data - Doc 0 is CLEARLY most relevant
test_query = "How do I cancel a flight booking?"

test_documents = [
    "To cancel a flight booking, log into your account, go to My Trips, select the booking, and click Cancel. You will receive a confirmation email within 24 hours.",  # MOST RELEVANT
    "Our refund policy allows full refunds within 24 hours of booking. After that, cancellation fees may apply depending on your fare type.",  # SOMEWHAT RELEVANT
    "To book a new flight, search for your destination, select dates, choose seats, and complete payment. You can pay with credit card or PayPal."  # NOT RELEVANT
]

print("Test data:")
print(f"Query: {test_query}")
print(f"\nDoc 0 (MOST relevant): {test_documents[0][:80]}...")
print(f"Doc 1 (somewhat relevant): {test_documents[1][:80]}...")
print(f"Doc 2 (NOT relevant): {test_documents[2][:80]}...")

In [ ]:
def test_reranker_format(payload: dict, format_name: str):
    """Test a specific input format"""
    print(f"\n{'='*60}")
    print(f"Testing format: {format_name}")
    print(f"{'='*60}")
    print(f"Payload keys: {payload.keys()}")
    
    try:
        response = reranker_client.invoke_endpoint(
            EndpointName=RERANKER_ENDPOINT,
            ContentType='application/json',
            Body=json.dumps(payload)
        )
        
        raw_bytes = response['Body'].read()
        print(f"Response length: {len(raw_bytes)} bytes")
        
        if len(raw_bytes) == 0:
            print("❌ Empty response")
            return None
        
        output = json.loads(raw_bytes.decode('utf-8'))
        print(f"✓ Got response")
        print(f"Response structure: {type(output)}")
        
        if isinstance(output, dict):
            print(f"Keys: {output.keys()}")
        
        return output
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

In [ ]:
# FORMAT 1: query + documents
payload1 = {
    "query": test_query,
    "documents": test_documents
}
result1 = test_reranker_format(payload1, "query + documents")

In [ ]:
# FORMAT 2: pairs
payload2 = {
    "pairs": [[test_query, doc] for doc in test_documents]
}
result2 = test_reranker_format(payload2, "pairs")

In [ ]:
# FORMAT 3: inputs (HuggingFace style)
payload3 = {
    "inputs": [[test_query, doc] for doc in test_documents]
}
result3 = test_reranker_format(payload3, "inputs (HF style)")

In [ ]:
# FORMAT 4: query + passages
payload4 = {
    "query": test_query,
    "passages": test_documents
}
result4 = test_reranker_format(payload4, "query + passages")

In [ ]:
# FORMAT 5: query + texts
payload5 = {
    "query": test_query,
    "texts": test_documents
}
result5 = test_reranker_format(payload5, "query + texts")

In [ ]:
# FORMAT 6: Qwen reranker specific format with document objects
payload6 = {
    "query": test_query,
    "documents": [{"text": doc} for doc in test_documents]
}
result6 = test_reranker_format(payload6, "query + document objects")

## Step 2: Analyze the Response

Use the format that worked above.

In [ ]:
# Use the format that worked (update this based on above results)
# For your response format, it looks like FORMAT 1 or 6 worked

working_result = result1  # <-- Change to the one that worked (result1, result2, etc.)

print("Full response:")
print(json.dumps(working_result, indent=2, default=str))

In [ ]:
# Extract and analyze scores
if working_result and 'results' in working_result:
    results = working_result['results']
    
    print("\nScore Analysis:")
    print("-" * 60)
    
    scores = []
    for r in results:
        idx = r['index']
        score = r['relevance_score']
        scores.append((idx, score))
        
        relevance = "MOST RELEVANT" if idx == 0 else ("SOMEWHAT" if idx == 1 else "NOT RELEVANT")
        print(f"  Doc {idx}: score = {score:.6f} ({relevance})")
    
    # Check score order
    scores_sorted = sorted(scores, key=lambda x: x[1], reverse=True)
    print(f"\nRanking by score (highest first): {[s[0] for s in scores_sorted]}")
    
    if scores_sorted[0][0] == 0:
        print("\n✓ CORRECT! Doc 0 (most relevant) has highest score")
    else:
        print(f"\n❌ WRONG! Doc {scores_sorted[0][0]} ranked first, should be Doc 0")
        
        # Check if lower is better
        scores_sorted_asc = sorted(scores, key=lambda x: x[1])
        if scores_sorted_asc[0][0] == 0:
            print("\n💡 TRY: Lower scores = better! Use score_order='asc'")

## Step 3: Test with Conversation History

This is where re-ranker should shine - understanding context.

In [ ]:
# Test: Ambiguous query WITH context
# The re-ranker should understand "documents" means cancellation documents

# Build a context-enriched query
history_context = "Previous conversation: User asked about cancelling a booking. User asked about refund policy."
ambiguous_query = "What documents do I need?"

# Option A: Prepend context to query
enriched_query_a = f"{history_context} Current question: {ambiguous_query}"

# Option B: Simpler - just key terms
enriched_query_b = f"cancellation refund booking - {ambiguous_query}"

# Option C: Question with context hint
enriched_query_c = f"Regarding booking cancellation and refunds: {ambiguous_query}"

print("Context-enriched queries:")
print(f"\nOption A (verbose): {enriched_query_a}")
print(f"\nOption B (keywords): {enriched_query_b}")
print(f"\nOption C (natural): {enriched_query_c}")

In [ ]:
# Test documents for ambiguous query
context_test_docs = [
    "To process a cancellation, you need: booking confirmation number, passenger ID, and the original payment receipt. Submit these via the cancellation form.",  # RELEVANT with context
    "Required documents for new bookings include: valid passport, credit card, and contact information.",  # NOT relevant with context
    "General documentation policy: All documents must be in PDF format and under 5MB.",  # NOT relevant
]

def test_with_query(query: str, docs: list, label: str):
    """Test re-ranker with a specific query"""
    payload = {
        "query": query,
        "documents": docs
    }
    
    response = reranker_client.invoke_endpoint(
        EndpointName=RERANKER_ENDPOINT,
        ContentType='application/json',
        Body=json.dumps(payload)
    )
    
    output = json.loads(response['Body'].read().decode('utf-8'))
    results = output['results']
    
    print(f"\n{label}")
    print(f"Query: {query[:80]}..." if len(query) > 80 else f"Query: {query}")
    print("-" * 50)
    
    scores = [(r['index'], r['relevance_score']) for r in results]
    scores_sorted = sorted(scores, key=lambda x: x[1], reverse=True)
    
    for idx, score in scores_sorted:
        print(f"  Rank: Doc {idx} (score: {score:.4f}) - {docs[idx][:50]}...")
    
    return scores_sorted[0][0]  # Return top-ranked doc index

In [ ]:
# Test without context (should maybe fail)
top_no_context = test_with_query(
    ambiguous_query, 
    context_test_docs, 
    "WITHOUT CONTEXT"
)

# Test with context Option A
top_context_a = test_with_query(
    enriched_query_a, 
    context_test_docs, 
    "WITH CONTEXT (Option A - verbose)"
)

# Test with context Option B  
top_context_b = test_with_query(
    enriched_query_b, 
    context_test_docs, 
    "WITH CONTEXT (Option B - keywords)"
)

# Test with context Option C
top_context_c = test_with_query(
    enriched_query_c, 
    context_test_docs, 
    "WITH CONTEXT (Option C - natural)"
)

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Expected top doc: 0 (cancellation documents)")
print(f"Without context: Doc {top_no_context} ranked first")
print(f"With context A:  Doc {top_context_a} ranked first")
print(f"With context B:  Doc {top_context_b} ranked first")
print(f"With context C:  Doc {top_context_c} ranked first")

## Step 4: Find the Best Context Format

Based on the above, identify which context format works best.

In [ ]:
# Determine best format
results_summary = {
    'no_context': top_no_context,
    'verbose': top_context_a,
    'keywords': top_context_b,
    'natural': top_context_c
}

correct = [k for k, v in results_summary.items() if v == 0]

if correct:
    print(f"✓ These context formats work: {correct}")
    print(f"\nRecommended: Use '{correct[0]}' format for building context-enriched queries")
else:
    print("❌ None of the formats produced correct ranking")
    print("\nPossible issues:")
    print("  1. Re-ranker model may not be suitable for your domain")
    print("  2. Document content may need preprocessing")
    print("  3. Consider using embedding-based approach instead")

## Step 5: Final Working Implementation

Based on what we learned, here's the optimized re-ranker.

In [ ]:
class OptimizedQwen3Reranker:
    """
    Optimized Qwen3-Reranker with proper context handling.
    """
    
    def __init__(self, endpoint_name: str, client):
        self.endpoint_name = endpoint_name
        self.client = client
        
        # Set based on your testing above
        self.context_format = 'natural'  # 'verbose', 'keywords', or 'natural'
    
    def build_contextualized_query(self, query: str, history: list) -> str:
        """
        Build query with conversation history context.
        """
        if not history:
            return query
        
        recent = history[-3:]  # Last 3 turns
        
        if self.context_format == 'verbose':
            # Detailed context
            context_parts = [f"User asked: {h.get('query', '')}" for h in recent]
            context = " ".join(context_parts)
            return f"Previous conversation: {context}. Current question: {query}"
        
        elif self.context_format == 'keywords':
            # Just key terms
            keywords = " ".join([h.get('query', '') for h in recent])
            return f"{keywords} - {query}"
        
        else:  # 'natural'
            # Natural phrasing
            topics = " and ".join([h.get('query', '').lower() for h in recent[-2:]])
            return f"Regarding {topics}: {query}"
    
    def rerank(self, query: str, documents: list, history: list = None) -> list:
        """
        Re-rank documents with optional conversation history.
        
        Returns: List of (index, score) tuples sorted by relevance
        """
        # Build contextualized query if history provided
        final_query = self.build_contextualized_query(query, history) if history else query
        
        # Call API
        payload = {
            "query": final_query,
            "documents": documents
        }
        
        response = self.client.invoke_endpoint(
            EndpointName=self.endpoint_name,
            ContentType='application/json',
            Body=json.dumps(payload)
        )
        
        output = json.loads(response['Body'].read().decode('utf-8'))
        results = output['results']
        
        # Extract scores sorted by index
        scores = [(r['index'], r['relevance_score']) for r in results]
        scores.sort(key=lambda x: x[0])  # Sort by index
        
        return [score for idx, score in scores]

print("✓ OptimizedQwen3Reranker class defined")

In [ ]:
# Test the optimized reranker
reranker = OptimizedQwen3Reranker(RERANKER_ENDPOINT, reranker_client)

# Set the context format that worked best (from Step 4)
reranker.context_format = 'natural'  # Update based on your results

# Test with history
test_history = [
    {'query': 'How do I cancel a booking?'},
    {'query': 'What is the refund policy?'}
]

scores = reranker.rerank(
    query="What documents do I need?",
    documents=context_test_docs,
    history=test_history
)

print("Final test with optimized reranker:")
print(f"Scores: {scores}")

# Rank documents
ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
print(f"Ranking: {ranked_indices}")
print(f"\nTop document: Doc {ranked_indices[0]}")
print(f"Expected: Doc 0 (cancellation documents)")

## Conclusion

If the re-ranker still doesn't work correctly after these tests:

1. **The model may not be well-suited** for your specific document domain
2. **Consider fine-tuning** the re-ranker on your data
3. **Use the embedding-based approach** with query expansion (which you already have)

But if one of the context formats worked, update your retriever to use it!

In [ ]:
print("="*60)
print("DEBUG COMPLETE")
print("="*60)
print("\nNext steps:")
print("1. Check which context format worked best above")
print("2. Update reranker.context_format in your code")
print("3. If none worked, use embedding-based query expansion")